# Import Packages

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import pandas as pd

# This is needed since the notebook is stored in the object_detection folder.

In [2]:
sys.path.append("..")

# Import utilites

In [4]:
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using

In [5]:
FOLDER = 'test_img'
OUT = 'out_img'

# Load images + test images on frozen model + visualize bounding box predictions on test images

In [7]:
pred = []
for filename in  os.listdir(FOLDER):

    MODEL_NAME = 'inference_graph'
    IMAGE_NAME = filename

    # Grab path to current working directory
    CWD_PATH = os.getcwd()

    # Path to frozen detection graph .pb file, which contains the model that is used
    # for object detection.
    PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

    # Path to label map file
    PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

    # Path to image
    PATH_TO_IMAGE = os.path.join(CWD_PATH, FOLDER, IMAGE_NAME)

    # Number of classes the object detector can identify maize and weed
    NUM_CLASSES = 2

    # Load the label map.
    # Label maps map indices to category names, so that when our convolution
    # network predicts `5`, we know that this corresponds to `king`.
    # Here we use internal utility functions, but anything that returns a
    # dictionary mapping integers to appropriate string labels would be fine
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    # Load the Tensorflow model into memory.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)

    # Define input and output tensors (i.e. data) for the object detection classifier

    # Input tensor is the image
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Output tensors are the detection boxes, scores, and classes
    # Each box represents a part of the image where a particular object was detected
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represents level of confidence for each of the objects.
    # The score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    # Number of objects detected
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Load image using OpenCV and
    # expand image dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    image = cv2.imread(PATH_TO_IMAGE)
    im_height, im_width, channels = image.shape
    #print(im_height, im_width, channels)
    image_expanded = np.expand_dims(image, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})

    # Draw the results of the detection (aka 'visulaize the results')

    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=5,
        min_score_thresh=0.60)

    min_score_thresh = 0.6
    
    score = scores[0]
    
    true_boxes = boxes[0][scores[0] > min_score_thresh]
    
    #generate predicted class names
    objects = []
    for index, value in enumerate(classes[0]):
        object_dict = {}
        if scores[0, index] > min_score_thresh:
            object_dict[(category_index.get(value)).get('name')] = \
                        scores[0, index]
            object_dict = next(iter(object_dict))
            objects.append(object_dict)
    

    #cv2.imshow('Object detector', image)
    #cv2.waitKey()
    #cv2.destroyAllWindows()
    
    # All the results have been drawn on image. Now write the image to the disk
    cv2.imwrite(os.path.join(CWD_PATH, OUT, IMAGE_NAME), image)
    
    #save prediction classes and predicted bounding boxes as dataframe to .csv format
    for i in range(true_boxes.shape[0]):
        
        ymin = true_boxes[i,0]*im_height
        xmin = true_boxes[i,1]*im_width
        ymax = true_boxes[i,2]*im_height
        xmax = true_boxes[i,3]*im_width

        pred.append((filename, im_width, im_height, objects[i], int(xmin), int(ymin), int(xmax), int(ymax)))
        
e = pd.DataFrame(pred, columns=('filename','width','height','class','xmin','ymin','xmax','ymax'))
e.to_csv('preds.csv',index=False)